<a href="https://colab.research.google.com/github/fabrizioaymone/machinelearning/blob/main/IALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os, sys
drive.mount("/content/drive")
sys.path.append("/content/drive/MyDrive/env")
os.symlink("/content/drive/MyDrive/datasets/H&M","/content/datasets")
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json

Mounted at /content/drive


In [3]:
import pandas as pd
users = pd.read_csv("/content/datasets/customers.csv")
items = pd.read_csv("/content/datasets/articles.csv")
URM = pd.read_csv("/content/datasets/transactions_train.csv")

In [4]:
users.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [5]:
items.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [45]:
user_ids = list(enumerate(users["customer_id"]))
item_ids = list(enumerate(items["article_id"]))
user_map = {user: uix for uix, user in user_ids}
item_map = {item: iix for iix, item in item_ids}

In [7]:
URM['user_map'] = URM["customer_id"].map(user_map)
URM['item_map'] = URM["article_id"].map(item_map)
URM = URM[['item_map', 'user_map']]


In [8]:
import scipy.sparse as sps
import numpy as np

URM = sps.coo_matrix((np.ones(len(URM)), (URM['user_map'], URM['item_map'])), shape=(len(users), len(items)))

In [9]:
train_test_split = 0.8
n_interactions = URM.nnz
train_mask = np.random.choice([True, False], n_interactions, p=[train_test_split, 1-train_test_split])
test_mask = np.invert(train_mask)

URM_train = sps.csr_matrix((URM.data[train_mask],(URM.row[train_mask], URM.col[train_mask])), shape=(len(users), len(items)))
URM_test = sps.csr_matrix((URM.data[test_mask],(URM.row[test_mask], URM.col[test_mask])), shape=(len(users), len(items)))

In [18]:
import implicit
from implicit import evaluation

def validate(csr_train, csr_test, factors, iterations, regularization, show_progress):
  model = implicit.als.AlternatingLeastSquares(factors=factors, iterations=iterations, regularization=regularization, use_gpu=True)
  model.fit(csr_train)
  map12 = evaluation.mean_average_precision_at_k(model, csr_train, csr_test, K=12, show_progress=show_progress)
  print("The model with factors={}, iterations={}, regularization={} has MAP@12={}".format(factors, iterations,regularization, map12))
  return map12

In [58]:
bestmap = 0
for factors in [ 175, 180, 185]:
  for iterations in [5, 6, 7]:
    for regularization in [0.5, 1, 1.5]:
      map = validate(URM_train, URM_test, factors, iterations, regularization, True)
      if map > bestmap:
        bestmap = map
        bestfactors = factors
        bestiterations = iterations
        bestregularization = regularization

print(f"BEST MAP: {bestmap}\n best factor: {bestfactors}\n best iterations: {bestiterations}\n best regularization: {bestregularization}")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1020997 [00:00<?, ?it/s]

The model with factors=170, iterations=4, regularization=0.5 has MAP@12=0.019733769792588497


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1020997 [00:00<?, ?it/s]

...

  0%|          | 0/1020997 [00:00<?, ?it/s]

The model with factors=175, iterations=6, regularization=0.5 has MAP@12=0.01992891688986945


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1020997 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [74]:
best_model = implicit.als.AlternatingLeastSquares(factors=180, iterations=6, regularization=1, use_gpu=True)
best_model.fit(URM.tocsr())
predictions, scores = best_model.recommend(np.arange(URM.shape[0]), URM.tocsr(), N=12, filter_already_liked_items=False)

  0%|          | 0/20 [00:00<?, ?it/s]

In [75]:
predictions

array([[16003, 78719, 78721, ..., 78939, 97668, 84170],
       [ 1469, 70221, 42626, ..., 33866, 15248, 13714],
       [ 1469, 59458, 89397, ...,  1475, 59460, 13713],
       ...,
       [ 4444, 53893, 17383, ..., 87395, 53897,  2255],
       [56694, 56692, 56693, ..., 74739, 89382, 40411],
       [72486,   958,  1425, ..., 60534, 91737, 13709]], dtype=int32)

In [76]:
preds = []
for i, pred in enumerate(predictions):
  userid = user_ids[i][1]
  itemids = ['0'+str(item_ids[item_id][1]) for item_id in pred]
  preds.append((userid, ' '.join(itemids)))

In [77]:
final = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
final.to_csv("/content/submission.csv", index=False)

In [78]:
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f submission.csv -m "No Message"

100% 258M/258M [00:10<00:00, 25.1MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations